In [27]:
%load_ext autoreload
%autoreload 2
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import eda

In [5]:
os.listdir('../input/')

['train.csv.zip',
 'merchants.csv.zip',
 'test.csv.zip',
 'Data_Dictionary.xlsx',
 'historical_transactions.csv.zip',
 'new_merchant_transactions.csv.zip',
 'sample_submission.csv.zip']

In [9]:
import glob
path_list = glob.glob('../input/*.csv')
df_list = []
for i, path in enumerate(path_list):
    df_list.append(pd.read_csv(path))
    print(df_list[i].shape)
    display(df_list[i].head())

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


,card_id,target
0,C_ID_0ab67a22ab,0
1,C_ID_130fd0cbdd,0
2,C_ID_b709037bc5,0
3,C_ID_d27d835a9f,0
4,C_ID_2b5e3df5c2,0


,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,1.666667,3,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,0.500000,3,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [28]:
is_pickle = True
is_base = False

def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')

if is_pickle:
    for df, path in zip(df_list, path_list):
        filename = re.search(r'/([^/.]*).csv', path).group(1)
        utils.to_df_pkl(path='../input/', fname=f"{filename}", df=df)

if is_base:
    train, test = quara_load_data()
    df = pd.concat([train, test], axis=0)
    utils.to_pkl_gzip(obj=df[['qid', 'target']], path='../input/base')
    sys.exit()

3it [00:00, 65.11it/s]
3it [00:00, 151.84it/s]
2it [00:00, 16.62it/s]

shape: (201917, 6)
shape: (123623, 2)
shape: (334696, 22)


3it [00:00, 16.56it/s]
0it [00:00, ?it/s]

shape: (1963031, 14)


3it [00:01,  2.14it/s]
3it [00:00, 113.80it/s]
0it [00:00, ?it/s]

shape: (123623, 5)
shape: (29112361, 14)


3it [00:22,  7.45s/it]


In [18]:
eda_result = pd.DataFrame()
for i, df in enumerate(df_list):
    filename = re.search(r'/([^/.]*).csv', path_list[i]).group(1)
    df = eda.df_info(df)
    df['filename'] = filename
    
    if len(eda_result):
        eda_result = pd.concat([eda_result, df], axis=0)
    else:
        eda_result = df.copy()
        
eda_result.to_csv(f'../eda/eda_df_info.csv', index=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Shape: (201917, 6)
Categorical: first_active_month
Categorical: card_id
Shape: (123623, 2)
Categorical: card_id
Shape: (334696, 22)
Categorical: merchant_id
Categorical: category_1
Categorical: most_recent_sales_range
Categorical: most_recent_purchases_range
Categorical: category_4
Shape: (1963031, 14)
Categorical: authorized_flag
Categorical: card_id
Categorical: category_1
Categorical: category_3
Categorical: merchant_id
Categorical: purchase_date
Shape: (123623, 5)
Categorical: first_active_month
Categorical: card_id
Shape: (29112361, 14)
Categorical: authorized_flag
Categorical: card_id
Categorical: category_1
Categorical: category_3
Categorical: merchant_id
Categorical: purchase_date


In [26]:
for df, path in zip(df_list, path_list):
    filename = re.search(r'/([^/.]*).csv', path).group(1)
    print(filename)
    print(df.shape)

train
(201917, 6)
sample_submission
(123623, 2)
merchants
(334696, 22)
new_merchant_transactions
(1963031, 14)
test
(123623, 5)
historical_transactions
(29112361, 14)
